In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

# --- STEP 1: LOAD DATA ---
ridership = pd.read_excel('Data/MARTA_Train_Ridership_2023_20250917.xlsx', sheet_name='Total trips by date, station') 
games = pd.read_csv('Data/atlanta_sports.csv')

ridership['Date'] = pd.to_datetime(ridership['Date'])
games['game_date'] = pd.to_datetime(games['game_date'])

# --- STEP 2: PREPARE VARIABLES (Month + Weekday) ---
# Filter for Stadium Stations
target_stations = ['Dome/GWCC', 'Vine City']
stadium_trips = ridership[ridership['Station'].isin(target_stations)].groupby('Date')['Total trips'].sum().reset_index()

# Create "Month" and "Day of Week" variables
stadium_trips['DayOfWeek'] = stadium_trips['Date'].dt.dayofweek
stadium_trips['Month'] = stadium_trips['Date'].dt.month

# Filter for Mon-Thu (0-3)
weekday_trips = stadium_trips[stadium_trips['DayOfWeek'] <= 3].copy()

# --- STEP 3: MERGE WITH GAMES ---
merged_df = pd.merge(weekday_trips, games, left_on='Date', right_on='game_date', how='left')
merged_df['is_mls'] = (merged_df['league'] == 'MLS').astype(int)

# --- STEP 4: RUN THE "BASE MODEL" (Month + Weekday + Game) ---
# We use 'C(Month)' to treat Month as a category (Jan, Feb, etc.)
# We use 'C(DayOfWeek)' to treat Mon, Tue, Wed as categories
X = merged_df[['is_mls', 'Month', 'DayOfWeek']]
# This creates the "dummy variables" for Month and Day automatically
X = pd.get_dummies(X, columns=['Month', 'DayOfWeek'], drop_first=True)
X = sm.add_constant(X) 
Y = merged_df['Total trips']

# Run the regression
model = sm.OLS(Y, X.astype(float)).fit()

# --- STEP 5: RESULTS ---
print(f"Base Ridership (Intercept): {model.params['const']:.0f}")
if 'is_mls' in model.params:
    print(f"MLS Game Impact: +{model.params['is_mls']:.0f} riders")
else:
    print("MLS impact variable not found in model params")

print("\n--- Full Model Summary ---")
print(model.summary())

Base Ridership (Intercept): 2079
MLS Game Impact: +3737 riders

--- Full Model Summary ---
                            OLS Regression Results                            
Dep. Variable:            Total trips   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     3.619
Date:                Tue, 02 Dec 2025   Prob (F-statistic):           4.91e-06
Time:                        16:19:47   Log-Likelihood:                -5047.8
No. Observations:                 566   AIC:                         1.013e+04
Df Residuals:                     550   BIC:                         1.020e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [4]:
# Check the Top 5 busiest days in your data
print(weekday_trips.sort_values('Total trips', ascending=False).head(5))

          Date   Total trips  DayOfWeek
225 2023-08-14  15041.162441          0
206 2023-07-26  13054.826178          2
921 2025-07-10  12852.071617          3
750 2025-01-20  12355.374524          0
543 2024-06-27  12351.835683          3


In [8]:
import pandas as pd
import statsmodels.api as sm

# --- STEP 1: LOAD DATA ---
# (Same as before)
ridership = pd.read_excel('ridership.xlsx', sheet_name='Total trips by date, station') 
games = pd.read_csv('Data/atlanta_sports.csv')

ridership['Date'] = pd.to_datetime(ridership['Date'])
games['game_date'] = pd.to_datetime(games['game_date'])

# --- STEP 2: PREPARE VARIABLES ---
# Filter for Stadium Stations
target_stations = ['Dome/GWCC', 'Vine City']
stadium_trips = ridership[ridership['Station'].isin(target_stations)].groupby('Date')['Total trips'].sum().reset_index()

# Create Month and Day variables
stadium_trips['DayOfWeek'] = stadium_trips['Date'].dt.dayofweek
stadium_trips['Month'] = stadium_trips['Date'].dt.month

# *** CHANGE IS HERE: WE DO NOT DELETE THE WEEKENDS ***
# We use the full 'stadium_trips' dataset instead of filtering for <= 3

# --- STEP 3: MERGE ---
# Merge the FULL dataset with games
full_week_df = pd.merge(stadium_trips, games, left_on='Date', right_on='game_date', how='left')
full_week_df['is_mls'] = (full_week_df['league'] == 'MLS').astype(int)

# --- STEP 4: RUN THE "FULL WEEK" MODEL ---
# We still control for DayOfWeek, so the math knows Sunday != Monday
X = full_week_df[['is_mls', 'Month', 'DayOfWeek']]
X = pd.get_dummies(X, columns=['Month', 'DayOfWeek'], drop_first=True)
X = sm.add_constant(X) 
Y = full_week_df['Total trips']

# Run regression
model_full = sm.OLS(Y, X.astype(float)).fit()

# --- STEP 5: RESULTS ---
print("--- FULL WEEK (Mon-Sun) RESULTS ---")
print(f"Baseline Ridership: {model_full.params['const']:.0f}")
print(f"MLS Game Impact: +{model_full.params['is_mls']:.0f} riders")
print(model_full.summary())

--- FULL WEEK (Mon-Sun) RESULTS ---
Baseline Ridership: 1497
MLS Game Impact: +4095 riders
                            OLS Regression Results                            
Dep. Variable:            Total trips   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     15.97
Date:                Tue, 02 Dec 2025   Prob (F-statistic):           1.08e-43
Time:                        16:30:37   Log-Likelihood:                -9401.9
No. Observations:                1006   AIC:                         1.884e+04
Df Residuals:                     987   BIC:                         1.894e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------